In [1]:
import numpy as np
import pandas as pd

In [2]:
import os

base_path = r"C:\Users\thaip\Desktop\Exercise 7\data\news_1135"

all_texts = []

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".txt"):  # chỉ đọc file .txt
            file_path = os.path.join(root, file)
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                content = f.read()
                all_texts.append({
                    "category": os.path.basename(root),  # tên thư mục cha (vd: Thể thao)
                    "filename": file,
                    "content": content
                })

print(f"Đã đọc {len(all_texts)} file văn bản.")
print("Ví dụ 1 file đầu tiên:\n")
print("Thể loại:", all_texts[0]["category"])
print("tên file:", all_texts[0]["filename"])
print("Nội dung:\n", all_texts[0]["content"][:500], "...")

Đã đọc 1135 file văn bản.
Ví dụ 1 file đầu tiên:

Thể loại: Giải trí
tên file: 00a00d8f63505813279de920ee6b0e364fdbf062.txt
Nội dung:
 Dân_trí Một quả_phụ đã đâm đơn_kiện cửa_hàng ở Mỹ vì tin rằng chồng bà đã bị stress đến chết sau khi bị bắt lau dọn_nhà_vệ_sinh ở đây . Theo bà Maria Elizarraras , người chồng 69 tuổi của bà đã chết vì trầm_cảm và stress sau khi nhân_viên tại một cửa_hàng của Walgreens cương_quyết không cho ông rời đi nếu_không chịu dọn_nhà_vệ_sinh của họ . Ảnh minh hoạÔng Elizarraras là khách quen của cửa_hàng và dùng nhà_vệ_sinh ở đây năm 2012 . Thế_nhưng các văn_bản tòa_án ghi lại rằng khi ông định rời đi , n ...


In [3]:
import pandas as pd
df = pd.DataFrame(all_texts)
df.to_csv("all_news.csv", index=False, encoding="utf-8-sig")

In [4]:
df.head()

,category,filename,content
0,Giải trí,00a00d8f63505813279de920ee6b0e364fdbf062.txt,Dân_trí Một quả_phụ đã đâm đơn_kiện cửa_hàng ở...
1,Giải trí,01ff19b7175627a2c4c9f5e934a1bdeb662a5755.txt,Dân_trí Hai người phụ_nữ từng là bạn gái của J...
2,Giải trí,0224086b1b54be3a42f93fd7d022b500e30af481.txt,Dân_trí Trong buổi ra_mắt sản_phẩm âm_nhạc mới...
3,Giải trí,049fd9dcd2241051ac6ca82f71255aa6d81a3e24.txt,"Mới_đây , Trịnh Kim_Chi xuất_hiện tại một sự_k..."
4,Giải trí,09ca4a9ae21d8e85dcac1ecea1910b7028f377af.txt,Dân_trí Cặp tình_nhân nổi_tiếng của màn_ảnh Ho...


In [5]:
df['category'].value_counts()


category
Khoa học - Công nghệ    196
Kinh tế                 186
Thể thao                140
Thời sự                 138
Giải trí                107
Tin kh†c                100
Đời sống - Xã hội        91
Sức khỏe                 75
Độc giả                  52
Pháp luật                50
Name: count, dtype: int64

In [6]:
from pyvi import ViTokenizer

In [7]:
df1 = df.drop(columns = ["filename"])
df1.head()

,category,content
0,Giải trí,Dân_trí Một quả_phụ đã đâm đơn_kiện cửa_hàng ở...
1,Giải trí,Dân_trí Hai người phụ_nữ từng là bạn gái của J...
2,Giải trí,Dân_trí Trong buổi ra_mắt sản_phẩm âm_nhạc mới...
3,Giải trí,"Mới_đây , Trịnh Kim_Chi xuất_hiện tại một sự_k..."
4,Giải trí,Dân_trí Cặp tình_nhân nổi_tiếng của màn_ảnh Ho...


In [8]:
with open("data/vietnamese-stopwords.txt", encoding="utf8") as f:
    stopwords = [x.strip().replace(" ", "_") for x in f.readlines()]

print("10 từ dừng đầu tiên:", stopwords[:10])

10 từ dừng đầu tiên: ['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']


In [10]:
from sklearn.svm import SVC
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

In [11]:
X = df["content"]
y = df["category"]

In [12]:
tfidf = TfidfTransformer()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df["content"], df["category"], test_size=0.2, random_state=30)
vectorizer = CountVectorizer(stop_words=stopwords)
tfidf = TfidfTransformer()
model_preprocess = Pipeline([
    ("vect", vectorizer),
    ("tfidf", tfidf),
])

X_tfidf = model_preprocess.fit_transform(X_train)

print("Shape TF-IDF:", X_tfidf.shape)

vocab = model_preprocess.named_steps["vect"].vocabulary_

print("\n10 từ đầu tiên trong từ điển:")
for i, (k, v) in enumerate(list(vocab.items())[:10]):
    print(f"{i+1}: ({k}, {v})")

Shape TF-IDF: (908, 21967)

10 từ đầu tiên trong từ điển:
1: (dân_trí, 6295)
2: (biến, 2836)
3: (đoạn, 20886)
4: (video, 19241)
5: (hài_hước, 8207)
6: (youtube, 20378)
7: (thành, 16568)
8: (ảnh, 21862)
9: (gif, 7124)
10: (động, 21750)


In [14]:
tf_vec= X_tfidf.toarray()
tf_vec[:10]

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02220944, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [16]:
model = SVC(kernel='linear', C=1)
model.fit(X_tfidf, y_train)

,C,1
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [17]:
X_test_tfidf = model_preprocess.transform(X_test)


In [18]:
y_pred = model.predict(X_test_tfidf)


In [19]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("🔹 Accuracy:", accuracy_score(y_test, y_pred))
print("\n🔹 Classification report:")
print(classification_report(y_test, y_pred))

🔹 Accuracy: 0.8370044052863436

🔹 Classification report:
                      precision    recall  f1-score   support

            Giải trí       0.70      0.88      0.78        16
Khoa học - Công nghệ       0.73      0.88      0.80        34
             Kinh tế       0.86      0.90      0.88        41
           Pháp luật       0.75      0.75      0.75         8
            Sức khỏe       0.86      0.60      0.71        20
            Thể thao       1.00      0.97      0.98        30
             Thời sự       1.00      0.75      0.86        36
            Tin kh†c       0.89      0.94      0.91        17
             Độc giả       1.00      0.70      0.82        10
   Đời sống - Xã hội       0.60      0.80      0.69        15

            accuracy                           0.84       227
           macro avg       0.84      0.82      0.82       227
        weighted avg       0.86      0.84      0.84       227



In [20]:
model_2 = SVC(kernel='rbf', C=1.0)
model_2.fit(X_tfidf, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [23]:
y_pred_rbf = model_2.predict(X_test_tfidf)


In [24]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("🔹 Accuracy:", accuracy_score(y_test, y_pred_rbf))
print("\n🔹 Classification report:")
print(classification_report(y_test, y_pred_rbf))

🔹 Accuracy: 0.801762114537445

🔹 Classification report:
                      precision    recall  f1-score   support

            Giải trí       0.72      0.81      0.76        16
Khoa học - Công nghệ       0.55      0.97      0.70        34
             Kinh tế       0.84      0.90      0.87        41
           Pháp luật       0.86      0.75      0.80         8
            Sức khỏe       1.00      0.50      0.67        20
            Thể thao       1.00      0.97      0.98        30
             Thời sự       1.00      0.67      0.80        36
            Tin kh†c       0.88      0.88      0.88        17
             Độc giả       1.00      0.50      0.67        10
   Đời sống - Xã hội       0.77      0.67      0.71        15

            accuracy                           0.80       227
           macro avg       0.86      0.76      0.79       227
        weighted avg       0.86      0.80      0.80       227



In [25]:
def predict(content):
    Vi_content = ViTokenizer.tokenize(content)
    Vi_input = vectorizer.transform([Vi_content])
    pred = model.predict(Vi_input)

    return pred

In [26]:
content = "khi chưa trả hết nợ số tiền vay_mượn chạy_chữa cho vợ thì bản_thân ông lại mắc căn_bệnh ung_thư quái_ác . Nhiều lần ra Hà_Nội điều_trị , cắt_bỏ dạ_dày khiến cơ_thể ông Ninh chỉ còn 40 kg . “ Năm 2015 , tôi phát_hiện bị ung_thư dạ_dày , ban_đầu đau bình_thường , sau đó đau quá nên cuối tháng 6 vừa_rồi tôi ra bệnh_viện K phẫu_thuật và cắt_bỏ luôn dạ_dày , vợ tôi bị ung_thư vú năm 2006 cũng đã phẫu_thuật cắt_bỏ 2 vú , bây_giờ sức_khỏe 2 vợ_chồng yếu không làm được gì chỉ ở nhà trông cháu giúp các con ” , ông Ninh cho biết . Theo các cụ cao_niên , trước_kia khu_vực chợ Hổ_Bái , thuộc thôn Phú_Đức là kho thuốc_trừ_sâu của hợp_tác_xã nông_nghiệp . Hơn chục năm trước , kho thuốc này mới được phá_bỏ , hợp_tác_xã được di_chuyển đi nơi khác để xây_dựng chợ . Ở xã Yên_Bái , người dân chủ_yếu dùng nguồn nước giếng_khơi và giếng_khoan để phục_vụ sinh_hoạt thường_ngày . Số người mắc bệnh ngày_càng tăng khiến bà_con lo_lắng và trở_thành nỗi ám_ảnh đối_với nhiều gia_đình "

In [27]:
A = predict(content)
A

array(['Sức khỏe'], dtype=object)